## 정보검색
1. Vector Space Model
2. Indexing
3. Inverted Document
4. Weighting
5. Retrieving


- TF/IDF
![7](https://user-images.githubusercontent.com/38183218/43502447-e3624b86-9595-11e8-895b-150ed944052c.jpg)

- Similarity
![8](https://user-images.githubusercontent.com/38183218/43502459-f96733f6-9595-11e8-8094-60a1791d8daa.jpg)
![9](https://user-images.githubusercontent.com/38183218/43502460-f991e916-9595-11e8-880c-6295e33fdf91.jpg)

#### 실습: 뉴스 검색 엔진을 만들어보자

In [58]:
import requests
from bs4 import BeautifulSoup
import glob
import re
import nltk
from konlpy.tag import Kkma
from math import log10, sqrt

In [2]:
resp = requests.get("http://media.daum.net")

In [6]:
soup = BeautifulSoup(resp.content,'lxml')

In [15]:
aList = soup.select('.list_headline  a.link_txt')

In [20]:
links = []
for a in aList:
    links.append(a.get('href'))

In [21]:
links

['http://v.media.daum.net/v/20180801141138247',
 'http://v.media.daum.net/v/20180801140951174',
 'http://v.media.daum.net/v/20180801140921160',
 'http://v.media.daum.net/v/20180801140802122',
 'http://v.media.daum.net/v/20180801140622059',
 'http://v.media.daum.net/v/20180801140406973',
 'http://v.media.daum.net/v/20180801140352961',
 'http://v.media.daum.net/v/20180801140009800',
 'http://v.media.daum.net/v/20180801140007797',
 'http://v.media.daum.net/v/20180801140007793',
 'http://v.media.daum.net/v/20180801140004784',
 'http://v.media.daum.net/v/20180801135922758']

In [39]:
news_docs = []
for link in links:
    resp = requests.get(link)
    news_soup = BeautifulSoup(resp.content,'lxml')
    
    title = news_soup.find('title')
    pars = news_soup.select('.article_view p ')
    article = ''.join([par.text for par in pars if len(par)>0])
    news_docs.append((title.text, article))

In [58]:
news_docs[0]

('강원도 홍천 40.3도..우리나라 기상관측 이래 역대 최고(2보) | Daum 뉴스',
 '(서울=연합뉴스) 김승욱 기자 = 강원도 홍천의 수은주가 1일 40.3도까지 치솟아 기상관측 이래 역대 최고기온을 갈아치웠다.기상청에 따르면 이날 오후 1시 59시께 40.1도를 기록한 뒤 오후 2시 1분께 40.3도로 기온이 더 올랐다.우리나라 기상관측 역대 최고 온도다.부산·인천 1904년, 서울 1907년 등 현대적인 기상관측 장비가 도입된 20세기 초반 이래 전국에서 40도를 돌파한 적은 1942년 8월 1일 대구(40.0도)가 유일했다.앞서 경북 의성은 올해 7월 27일 39.9도, 충북 추풍령은 1939년 7월 21일 39.8도를 기록한 바 있다.ksw08@yna.co.kr')

In [41]:
len(news_docs)

12

In [52]:
for i in range(len(news_docs)):
    with open('docs/news_docs{0}.txt'.format(i),'w',encoding='utf-8') as fp:
        fp.write(news_docs[i][0]+'\n'+news_docs[i][1])

In [3]:
fileList = glob.glob('/Users/Chankoo/Desktop/GitHub/python/docs/*')

In [11]:
for file in fileList[:1]: 
    with open(file,'r',encoding='utf-8') as fp:
        content = fp.read()
        content = re.sub(r"[\s]{2,}","",content) # strip
        for sentence in nltk.sent_tokenize(content):
            words = nltk.regexp_tokenize(sentence, r"[0-9ㄱ-ㅎㅏ-ㅣ가-힛]+") # 한글 tokenizing을 위해 regexp를 쓴다
            for word in words:
                tokens = []
                for i in range(len(word)):
                    if i+1<len(word): # 1음절, 마지막음절 제거
                        tokens.append(word[i:i+2]) # bigram으로 자름
                print(word)
                print(tokens, end='\n')

강원도
['강원', '원도']
홍천
['홍천']
40
['40']
3도
['3도']
우리나라
['우리', '리나', '나라']
기상관측
['기상', '상관', '관측']
이래
['이래']
역대
['역대']
최고
['최고']
2보
['2보']
뉴스
['뉴스']
서울
['서울']
연합뉴스
['연합', '합뉴', '뉴스']
김승욱
['김승', '승욱']
기자
['기자']
강원도
['강원', '원도']
홍천의
['홍천', '천의']
수은주가
['수은', '은주', '주가']
1일
['1일']
40
['40']
3도까지
['3도', '도까', '까지']
치솟아
['치솟', '솟아']
기상관측
['기상', '상관', '관측']
이래
['이래']
역대
['역대']
최고기온을
['최고', '고기', '기온', '온을']
갈아치웠다
['갈아', '아치', '치웠', '웠다']
기상청에
['기상', '상청', '청에']
따르면
['따르', '르면']
이날
['이날']
오후
['오후']
1시
['1시']
59시께
['59', '9시', '시께']
40
['40']
1도를
['1도', '도를']
기록한
['기록', '록한']
뒤
[]
오후
['오후']
2시
['2시']
1분께
['1분', '분께']
40
['40']
3도로
['3도', '도로']
기온이
['기온', '온이']
더
[]
올랐다
['올랐', '랐다']
우리나라
['우리', '리나', '나라']
기상관측
['기상', '상관', '관측']
역대
['역대']
최고
['최고']
온도다
['온도', '도다']
부산
['부산']
인천
['인천']
1904년
['19', '90', '04', '4년']
서울
['서울']
1907년
['19', '90', '07', '7년']
등
[]
현대적인
['현대', '대적', '적인']
기상관측
['기상', '상관', '관측']
장비가
['장비', '비가']
도입된
['도입', '입된']
20세기
['20', '0세', '세기']
초반
['초반']
이래
['이래']
전국에서
['전국', '국

#### Forward Indexing
![29](https://user-images.githubusercontent.com/38183218/43558021-5154a3e4-9642-11e8-8194-274bd2dc9a29.png)

In [9]:
unq_nouns = [] # vetorize를 위해 unique한 noun을 뽑아낸다

for sentence in nltk.sent_tokenize(content):
    nouns = []
    for word in nltk.word_tokenize(sentence):
        nouns.extend(Kkma().nouns(word))
        nouns = list(set(nouns))
    unq_nouns.extend(nouns)

unq_nouns = list(set(unq_nouns))
print(len(unq_nouns)) # 차원의 수

98


In [13]:
#docs 전체의 unq_nouns를 만들자
for file in fileList:
     with open(file,'r',encoding='utf-8') as fp:
        content = fp.read()
        content = re.sub(r"[\s]{2,}","",content) # strip
        
        for sentence in nltk.sent_tokenize(content):
            nouns = []
            for word in nltk.word_tokenize(sentence):
                nouns.extend(Kkma().nouns(word))
                nouns = list(set(nouns))
            
            unq_nouns.extend(nouns)
            unq_nouns = list(set(unq_nouns))
            
print(len(unq_nouns)) # 차원의 수

2053


In [17]:
doc_nouns = {}
maxCount = {}

for file in fileList:
    docid = file[-17:]
    maxCount[docid] = 0
    with open(file,'r',encoding='utf-8') as fp:
        content = fp.read()
        content = re.sub(r"[\s]{2,}","",content) # strip
        
    nouns = {}
    
    for sentence in nltk.sent_tokenize(content): 
        for word in nltk.word_tokenize(sentence):
            for w in Kkma().nouns(word):
                if w in nouns.keys():
                    nouns[w] += 1
                else:
                    nouns[w] = 1
                    
                if maxCount[docid] < nouns[w]:
                    maxCount[docid] = nouns[w]
                    
    doc_nouns[docid] = nouns


In [18]:
for k,v in doc_nouns.items():
    print("{0} - {1}/{2}".format(k, len(v), maxCount[k])) # 문서id-   등장단어수/가장 높았던 빈도

cs\news_docs0.txt - 98/6
cs\news_docs1.txt - 185/22
s\news_docs10.txt - 402/40
s\news_docs11.txt - 371/17
cs\news_docs2.txt - 135/9
cs\news_docs3.txt - 313/30
cs\news_docs4.txt - 123/7
cs\news_docs5.txt - 16/1
cs\news_docs6.txt - 163/7
cs\news_docs7.txt - 269/19
cs\news_docs8.txt - 463/24
cs\news_docs9.txt - 221/12


In [24]:
doc_nouns.keys()

dict_keys(['cs\\news_docs0.txt', 'cs\\news_docs1.txt', 's\\news_docs10.txt', 's\\news_docs11.txt', 'cs\\news_docs2.txt', 'cs\\news_docs3.txt', 'cs\\news_docs4.txt', 'cs\\news_docs5.txt', 'cs\\news_docs6.txt', 'cs\\news_docs7.txt', 'cs\\news_docs8.txt', 'cs\\news_docs9.txt'])

In [23]:
doc_nouns['cs\\news_docs1.txt'] # 문서에 등장 단어와 빈도를 딕셔너리로 저장

{'1': 3,
 '1265': 1,
 '1265억원': 1,
 '1300': 1,
 '190': 1,
 '190억': 1,
 '1억1300만': 1,
 '21': 1,
 '21조': 1,
 '3': 1,
 '30': 2,
 '30년간': 1,
 '30일': 1,
 '3국': 1,
 '620': 3,
 '620억': 2,
 '620억달러': 1,
 '68': 1,
 '68조원': 1,
 '가운데': 1,
 '간': 1,
 '간섭': 1,
 '간여': 1,
 '개': 1,
 '개선': 1,
 '건설': 7,
 '격돌': 2,
 '견제': 2,
 '결국': 1,
 '경': 1,
 '경우': 1,
 '경제': 1,
 '고': 1,
 '과': 1,
 '관계': 1,
 '관심': 1,
 '구제': 7,
 '구제금융': 2,
 '국': 1,
 '국무': 1,
 '국무장관': 1,
 '국방장관': 1,
 '국제': 1,
 '국제통화기금': 1,
 '금융': 7,
 '기금': 1,
 '기술': 1,
 '기자': 1,
 '년': 1,
 '눈길': 1,
 '뉴스': 2,
 '뉴스1': 1,
 '니스': 1,
 '다대': 1,
 '다대다로': 1,
 '다로': 1,
 '달러': 5,
 '대': 1,
 '대부분': 1,
 '대테러': 1,
 '더': 2,
 '도': 1,
 '도로': 1,
 '돈': 2,
 '두': 1,
 '두고': 2,
 '등': 1,
 '등에': 1,
 '때': 1,
 '때문': 1,
 '로': 1,
 '를': 3,
 '만': 1,
 '매티스': 1,
 '무역': 1,
 '무역전쟁': 1,
 '미': 2,
 '미국': 22,
 '미국판': 2,
 '미중': 1,
 '박': 1,
 '박형기': 1,
 '반대': 3,
 '반응': 1,
 '발언': 1,
 '발전소': 2,
 '방해': 1,
 '법': 1,
 '변화': 1,
 '분쟁': 1,
 '비난': 1,
 '사업': 4,
 '사이': 1,
 '서로': 1,
 '서울': 1,
 '세력권': 1,
 '수': 2,


#### Inverted Indexing

![28](https://user-images.githubusercontent.com/38183218/43557954-1fcd53de-9642-11e8-9c8e-bb5f9aebc331.jpg)
![30](https://user-images.githubusercontent.com/38183218/43557955-1ffc6a0c-9642-11e8-97ed-6e2f78c07d8b.png)

In [25]:
invertedIdx = {}
for noun in unq_nouns:
    invertedIdx[noun] = []
    
    for k,v in doc_nouns.items():
        if noun in v:
            invertedIdx[noun].append(k)

In [27]:
print(invertedIdx['구제금융'], invertedIdx['강원도']) #단어기준으로 문서찾는 inverted 인덱싱이 완료됨

['cs\\news_docs1.txt'] ['cs\\news_docs0.txt', 'cs\\news_docs2.txt', 'cs\\news_docs5.txt']


#### Weighting

tf와 idf 모두 보정이 필요, 문서 수 가 적은 경우에는 tf만 보정하기도 한다
![34](https://user-images.githubusercontent.com/38183218/43563535-8e714410-965d-11e8-998d-ee8c8da5957a.png)

In [30]:
K = 0.5 # K + (1-K)*(f(t,d)/maxf(t,d)) 최솟값 0.5, 최댓값 1 
TF = {}
for k,v in doc_nouns.items():
    tfList = {}
    
    for w in v:
        tfList[w] =  K + (1-K)*(v[w]/maxCount[k])
#         print('{0} | {1} + {2} *({3}/{4})  =  {5}'.format(
#             w, K, (1-K), v[w], maxCount[k], tfList[w]
#         ))
        
    TF[k] = tfList

In [36]:
docSize = len(fileList)
TFIDF = {}
"""
TF = doc1|{단어1:0.54, 단어2:056 ...} 
"""
for (k,v) in TF.items():
    idfList = {}
    for t in v:
        idf = log10(docSize/len(invertedIdx[t]))
        idfList[t] = v[t] * idf
#         print('{0} | {1} * log({2} / {3}) = {4}'.format(
#             t, v[t], docSize, len(invertedIdx[t]), idfList[t]
#         ))
    
    TFIDF[k] = idfList

In [77]:
sortedByWeight = [sorted(dic.items(), key=lambda kv:kv[1],reverse=True) for dic in TFIDF.values()] # 내림차순 정렬해보자

In [79]:
sortedByWeight[1][:10] #1번 doc의 TFIDF기반 중요단어 top10을 뽑았다

[('미국', 1.0791812460476249),
 ('파키스탄', 1.03012755304546),
 ('구제', 0.711278548531389),
 ('건설', 0.711278548531389),
 ('일대일로', 0.637698009028142),
 ('일로', 0.637698009028142),
 ('인프라', 0.6131711625270595),
 ('620', 0.6131711625270595),
 ('일대일', 0.6131711625270595),
 ('미', 0.5886443160259772)]

#### Retrieving

- Cosine similarity: 벡터의 방향을 기준으로 similarity도출, 1사분면만 쓰기에(음수값없음) 0~1 사이의 값을 가짐
![39](https://user-images.githubusercontent.com/38183218/43562271-946b5c7c-9656-11e8-8f6c-221b5602d2dc.png)

B vector는 쿼리를 의미, 모든 경우에 다 필요하므로 연산안해도 된다

쿼리를 받아서 똑같이 전처리 한 뒤, tf-idf매긴다... AdHoc모델

In [55]:
query = '미국과 중국 중 미국 이겨라'

query_nouns = {}
maxCount = 0
    
for word in nltk.word_tokenize(query):
    for w in Kkma().nouns(word):
        if w in query_nouns.keys():
            query_nouns[w] += 1
        else:
            query_nouns[w] = 1
                    
        if maxCount < query_nouns[w]:
            maxCount = query_nouns[w]
    
print(query_nouns)

{'미국': 2, '중국': 1, '중': 1, '라': 1}


쿼리를 한 doc으로 보고 가중치를 구한다, 여기서는 tf만 구했다

In [57]:
query_weight = {}

for k,v in query_nouns.items():
    query_weight[k] = K + (1-K) * (v/maxCount)
    print('{0} | {1} + {2} *({3}/{4})  =  {5}'.format(
            k, K, (1-K), v, maxCount, query_weight[k]
        ))

미국 | 0.5 + 0.5 *(2/2)  =  1.0
중국 | 0.5 + 0.5 *(1/2)  =  0.75
중 | 0.5 + 0.5 *(1/2)  =  0.75
라 | 0.5 + 0.5 *(1/2)  =  0.75


doc의 len(문서별 단어 가중치의 유클리디언 합) 구해야한다

In [59]:
doc_len = {} #문서1:길이, 문서2:길이 ...

for k,v in TFIDF.items(): #k는 문서id, v는 단어를 키, 가중치를 밸류로 가진 딕셔너리
    sumPow = 0
    
    for t in v:
        sumPow += v[t]**2
    
    doc_len[k] = sqrt(sumPow)
    

In [60]:
print(doc_len)

{'cs\\news_docs0.txt': 4.899413355380067, 'cs\\news_docs1.txt': 6.558866887131949, 's\\news_docs10.txt': 10.312014879907268, 's\\news_docs11.txt': 10.15287536836188, 'cs\\news_docs2.txt': 5.788706649529996, 'cs\\news_docs3.txt': 8.366022889355873, 'cs\\news_docs4.txt': 5.960473392924283, 'cs\\news_docs5.txt': 2.1381360132336273, 'cs\\news_docs6.txt': 7.023185415453959, 'cs\\news_docs7.txt': 8.33759189891698, 'cs\\news_docs8.txt': 11.194428670049708, 'cs\\news_docs9.txt': 7.822790935307187}


In [84]:
result = {}
for k,v in query_weight.items():
    if k in invertedIdx.keys():
        print(k)

        for docid in invertedIdx[k]:
            if docid in result.keys():
                result[docid] += TFIDF[docid][k] * query_weight[k]
            else:
                result[docid] = TFIDF[docid][k] * query_weight[k]

            print('docid:{0}, weight:{1}'.format(docid,TFIDF[docid][k]))
            
print('\n',result)

미국
docid:cs\news_docs1.txt, weight:1.0791812460476249
중국
docid:cs\news_docs1.txt, weight:0.4445902600796855
docid:s\news_docs10.txt, weight:0.298200784199789
docid:cs\news_docs3.txt, weight:0.24651264827182562
docid:cs\news_docs8.txt, weight:0.2783207319198031
중
docid:cs\news_docs1.txt, weight:0.26024795711981585
docid:s\news_docs10.txt, weight:0.2504886587278228
docid:cs\news_docs6.txt, weight:0.47712125471966244
docid:cs\news_docs9.txt, weight:0.25844067963981715
라
docid:s\news_docs10.txt, weight:0.40852940645141295
docid:cs\news_docs6.txt, weight:0.4446578573620821

 {'cs\\news_docs1.txt': 1.6078099089472508, 's\\news_docs10.txt': 0.7179141370342685, 'cs\\news_docs3.txt': 0.1848844862038692, 'cs\\news_docs8.txt': 0.20874054893985233, 'cs\\news_docs6.txt': 0.6913343340613084, 'cs\\news_docs9.txt': 0.19383050972986288}


In [98]:
result.sort()

for k,v in result:
    print("[{0}] 유사도:{1}".format(k,v)) # 해당문서와 쿼리간의 정렬된 코사인 유사도를 도출

[cs\news_docs1.txt] 유사도:0.005698338577228374
[cs\news_docs3.txt] 유사도:0.0003157499753311188
[cs\news_docs6.txt] 유사도:0.0019956561901775358
[cs\news_docs8.txt] 유사도:0.0001487993455287504
[cs\news_docs9.txt] 유사도:0.00040488992135811976
[s\news_docs10.txt] 유사도:0.0006546993540152517


In [103]:
with open(fileList[1],encoding='utf-8') as fp:
    print(fp.read()) #쿼리와 가장 유사도가 높은 뉴스이다, 대충맞다!

미중 분쟁 점입가경, 이번에는 파키스탄 두고 격돌 | Daum 뉴스
(서울=뉴스1) 박형기 기자 = 최근 무역전쟁을 벌이고 있는 미국과 중국이 이번에는 파키스탄을 두고 격돌하고 있다.파키스탄이 미국이 주도하고 있는 국제통화기금(IMF)에 구제금융 신청하자 미국이 이를 반대하고 나선 것. 파키스탄이 IMF의 구제 금융을 받으면 그 돈이 모두 중국으로 흘러들어갈 것이란 이유에서다.◇ 미국, 파키스탄 IMF 구제금융 반대 : 실제 중국은 일대일로를 건설하면서 파키스탄에 막대한 투자를 하고 있다. 파키스탄은 바로 중동과 연결되기 때문에 일대일로에서 전략적인 위치를 차지하고 있다. 중국은 그런 파키스탄의 인프라 건설을 위해 620억 달러(68조원)를 차관 등의 형태로 제공하고 있다.미국은 파키스탄이 IMF의 구제 금융을 받으면 그 돈이 결국은 채권자인 중국에 흘러들어갈 것이란 이유로 IMF 구제 금융을 반대하고 있는 것이다.그러나 파키스탄은 미국이 중국과 파키스탄의 사이를 벌리려 하고 있다고 비난하고 나섰다.파키스탄은 원래 미국이 아프가니스탄과 대테러 전쟁을 펼칠 때, 미국과 좋은 관계를 유지했으나 최근 중국이 파키스탄에 막대한 투자를 함에 따라 중국과의 관계가 급속히 개선되고 있다.◇ 중국-파키스탄 620억달러 CPEC 사업 추진 : 중국은 현재 파키스탄과 총 620억 달러에 달하는 ‘중국 파키스탄 경제 회랑(CPEC)’ 건설 사업을 추진하고 있다. 이 가운데 190억 달러(약 21조 원)의 도로 및 발전소 건설 사업을 진행 중이거나 완료했다.이들 인프라 건설 프로그램은 대부분 중국 측의 차관으로 건설된다. 그러나 발전소의 경우, 완공 후 30년간 파키스탄이 전력을 사들이는 조건으로 건설되고 있다. 파키스탄 정부는 IMF 구제 금융과 CPEC를 연계하려는 미국의 의도에 불쾌한 반응을 보이고 있다. 따라서 파키스탄 재무부는 IMF 구제 금융과 CPEC을 연계하는 것은 터무니없는 주장이며, 미국 같은 제 3국이 이에 간여해서는 안 될 것이라고 주장하고 있다.중국도 미국의

언어처리 모델도 모델, 성능평가 해야한다

언어처리하는 모델에서는 accuracy 전혀 무의미, precision과 recall을 봐야함
![44](https://user-images.githubusercontent.com/38183218/43564599-68e8031e-9662-11e8-8c4b-10d1946b213e.png)